In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np

## Read data from csv

In [31]:
df = pd.read_csv("./processed/val.csv")
df["tokenized"] = df["tokenized"].apply(eval)
df.head()

,0,tokenized,cleaned
0,( 27 ) قَوْلُهُ : وَلَا تُكْرَهُ ضِيَافَتُهُ .\n,"[قوله, ولا, تكره, ضيافته]",قوله ولا تكره ضيافته
1,( الْفَرْقُ الثَّالِثُ وَالثَّلَاثُونَ بَيْنَ ...,"[الفرق, الثالث, والثلاثون, بين, قاعدة, تقدم, ا...",الفرق الثالث والثلاثون بين قاعدة تقدم الحكم عل...
2,( قَوْلُهُ : وَهُوَ ) أَيْ : الْبَيْعُ بِالْمَ...,"[قوله, وهو, أي, البيع, بالمعنى, الثاني, الذي, ...",قوله وهو أي البيع بالمعنى الثاني الذي هو العقد...
3,وَالْعَفْوُ قَبْلَ الْإِمَامِ ، أَوْ بَعْدَهُ ...,"[والعفو, قبل, الإمام, ،, أو, بعده, ،, إن, أراد...",والعفو قبل الإمام ، أو بعده ، إن أراد سترا
4,( قَوْلُهُ : وَرِبْحُهُ ) أَيْ الْقِرَاضِ وَقَ...,"[قوله, وربحه, أي, القراض, وقوله, كخراجه, أي, أ...",قوله وربحه أي القراض وقوله كخراجه أي أجرة خدمت...


## Extract Features

#### TF-IDF

In [32]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_vectorizer.fit(df['cleaned'])

TfidfVectorizer()

In [33]:
def extract_features(sent):
    return tfidf_vectorizer.transform([sent]).todense()

df["tf_idf"] = df["cleaned"].apply(extract_features)
df.head()

,0,tokenized,cleaned,tf_idf
0,( 27 ) قَوْلُهُ : وَلَا تُكْرَهُ ضِيَافَتُهُ .\n,"[قوله, ولا, تكره, ضيافته]",قوله ولا تكره ضيافته,[[[[[0. 0. 0. ... 0. 0. 0.]]]]]
1,( الْفَرْقُ الثَّالِثُ وَالثَّلَاثُونَ بَيْنَ ...,"[الفرق, الثالث, والثلاثون, بين, قاعدة, تقدم, ا...",الفرق الثالث والثلاثون بين قاعدة تقدم الحكم عل...,[[[[[0. 0. 0. ... 0. 0. 0.]]]]]
2,( قَوْلُهُ : وَهُوَ ) أَيْ : الْبَيْعُ بِالْمَ...,"[قوله, وهو, أي, البيع, بالمعنى, الثاني, الذي, ...",قوله وهو أي البيع بالمعنى الثاني الذي هو العقد...,[[[[[0. 0. 0. ... 0. 0. 0.]]]]]
3,وَالْعَفْوُ قَبْلَ الْإِمَامِ ، أَوْ بَعْدَهُ ...,"[والعفو, قبل, الإمام, ،, أو, بعده, ،, إن, أراد...",والعفو قبل الإمام ، أو بعده ، إن أراد سترا,[[[[[0. 0. 0. ... 0. 0. 0.]]]]]
4,( قَوْلُهُ : وَرِبْحُهُ ) أَيْ الْقِرَاضِ وَقَ...,"[قوله, وربحه, أي, القراض, وقوله, كخراجه, أي, أ...",قوله وربحه أي القراض وقوله كخراجه أي أجرة خدمت...,[[[[[0. 0. 0. ... 0. 0. 0.]]]]]


## Save features to csv

In [34]:
df.to_csv("./processed/val.csv", index=False)

In [40]:
df["tf_idf"][1].sum()

8.154773690669971

# pre and post characters

In [25]:
def create_input_vectors(text, window_size=1):
    """
    Creates input vectors for a character-level LSTM model,
    incorporating pre and post character features.

    Args:
        text (str): The text to be processed.
        window_size (int, optional): The number of characters to include
            on each side of the target character. Defaults to 1.

    Returns:
        list: A list of input vectors, where each vector is a NumPy array
            representing the target character, its previous characters,
            and its following characters.
    """

    input_vectors = []
    for i in range(len(text)):
        start = max(0, i - window_size)
        end = min(len(text), i + window_size + 1)
        context = text[start:end]
        target_char = text[i]
        input_vector = np.zeros(3 * (window_size * 2 + 1))  # Assuming one-hot encoding
        input_vector[window_size * 2] = 1  # Indicate target character
        # Set one-hot encodings for surrounding characters (replace with embeddings if needed)
        for j, char in enumerate(context):
            index = window_size - j if j < window_size else window_size + j + 1
            input_vector[index] = 1
        input_vectors.append(input_vector)
    return input_vectors

def create_bef_after(text):
    input_vectors = []
    for i in range(len(text)):
        letter_vec = []
        if i == 0:
            letter_vec.append(36)
            letter_vec.append(arabic_letters.index(text[i+1]))
            input_vectors.append(letter_vec)
            continue
        elif i == len(text)-1:
            letter_vec.append(arabic_letters.index(text[i-1]))

            letter_vec.append(36)
            input_vectors.append(letter_vec)
            break
        letter_vec.append(arabic_letters.index(text[i-1]))

        letter_vec.append(arabic_letters.index(text[i+1]))
        input_vectors.append(letter_vec)
    
    
    return input_vectors

In [30]:
def diacritics_probability_per_char(tokenized_cleaned, diacritics): 
    probability = {}
    
    for i in range(36):
        probability[i] = {}
        for j in range(15):
            probability[i][j] = 0
            
    counts = [0]*36
            
    for i in range(tokenized_cleaned):
        for j in range(len(tokenized_cleaned[i])):
            probability[arabic_letters.index(tokenized_cleaned[i][j])][diacritics[i][j]] += 1
            counts[arabic_letters.index(tokenized_cleaned[i][j])] += 1
            
            
    for i in range(36):
        for j in range(15):
            probability[i][j] /= counts[i]
        
    return probability

In [ ]:
import pickle
arabic_letters = list(pickle.load(
        open("./assets/arabic_letters.pickle", "rb")))
arabic_letters

In [26]:
create_bef_after("والثلاثون")

[[36, 33],
 [25, 11],
 [33, 2],
 [11, 11],
 [2, 33],
 [11, 2],
 [33, 25],
 [2, 30],
 [25, 36]]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
